In [1]:
import cv2
from moviepy.editor import VideoFileClip
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [2]:
def RGB2GRAY_func(frame1):
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_RGB2GRAY)
    return gray1
 

In [3]:
def GaussianBlur_func(gray,kernel_size):
    blur_gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
    return blur_gray

In [4]:
def Canny_func(blur_gray, low_threshold, high_threshold):
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    return edges

In [5]:
def FillPoly_func(edges, mask, vertices, ignore_mask_color):
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)
    return masked_edges


In [6]:
def HoughLinesP_func(masked_edges, rho, theta, threshold,min_line_length, max_line_gap):
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)
    return lines

In [7]:
def main():
    cap = cv2.VideoCapture("solidYellowLeft.mp4")

    while(True):
        # Take each frame
        cv2.waitKey(100)
        ret, frame = cap.read()
        if ret == True:
            gray = RGB2GRAY_func(frame)
            # Define a kernel size and apply Gaussian smoothing
            kernel_size = 9
            blur_gray = GaussianBlur_func(gray,kernel_size)
            # Define our parameters for Canny and apply
            low_threshold = 50
            high_threshold = 150
            edges = Canny_func(blur_gray, low_threshold, high_threshold)

            mask = np.zeros_like(edges)
            ignore_mask_color = 255

            # This time we are defining a four sided polygon to mask
            imshape = frame.shape
            #vertices = np.array([[(0, imshape[0]), (390, 320), (540, 320), (imshape[1], imshape[0])]], dtype=np.int32)
            vertices = np.array([[(0, imshape[0]), (378, 344), (540, 320), (imshape[1], imshape[0])]], dtype=np.int32)

            masked_edges = FillPoly_func(edges, mask, vertices, ignore_mask_color)

            # Define the Hough transform parameters
            # Make a blank the same size as our image to draw on
            rho = 1 # distance resolution in pixels of the Hough grid
            theta = np.pi / 180  # angular resolution in radians of the Hough grid
            threshold = 5  # minimum number of votes (intersections in Hough grid cell)
            min_line_length = 2 # minimum number of pixels making up a line
            max_line_gap = 90  # maximum gap in pixels between connectable line segments
            line_image = np.copy(frame) * 0  # creating a blank to draw lines on

            # Run Hough on edge detected image
            # Output "lines" is an array containing endpoints of detected line segments

            lines = HoughLinesP_func(masked_edges, rho, theta, threshold,min_line_length, max_line_gap)

            # Iterate over the output "lines" and draw lines on a blank image
            for line in lines:
                for x1, y1, x2, y2 in line:
                    cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 4)

            # Create a "color" binary image to combine with line image
            color_edges = np.dstack((edges, edges, edges))

            # Draw the lines on the edge image
            lines_edges = cv2.addWeighted(frame, 0.7, line_image, 1, 0)
            cv2.imshow('frame',frame)
            cv2.imshow('lines_edges',lines_edges)
                
            # Check end of video
            if cv2.waitKey(1) & 0xFF == ord('q'):
                 break

        else:
            break
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()


In [8]:
if __name__ == '__main__':
    main() 